Tabular Classification

In [ ]:
!pip install opendatasets --quiet
import opendatasets as od
od.download("https://www.kaggle.com/datasets/mssmartypants/rice-type-classification")

In [ ]:
import os
os.listdir("/content/rice-type-classification")


In [ ]:
import pandas as pd # Data reading and preprocessing


In [ ]:
data_df = pd.read_csv("/content/rice-type-classification/riceClassification.csv")
data_df.head()


In [ ]:
data_df.dropna(inplace=True)
data_df.drop(["id"], axis=1, inplace= True)  # Drop Id column # axis=0 row-wise operation # axis=1 column-wise operation       
data_df.head()

In [ ]:
	Area	MajorAxisLength	MinorAxisLength	Eccentricity	ConvexArea	EquivDiameter	Extent	Perimeter	Roundness	AspectRation	Class
0	4537	92.229316	64.012769	0.719916	4677	76.004525	0.657536	273.085	0.764510	1.440796	1
1	2872	74.691881	51.400454	0.725553	3015	60.471018	0.713009	208.317	0.831658	1.453137	1
2	3048	76.293164	52.043491	0.731211	3132	62.296341	0.759153	210.012	0.868434	1.465950	1
3	3073	77.033628	51.928487	0.738639	3157	62.551300	0.783529	210.657	0.870203	1.483456	1
4	3693	85.124785	56.374021	0.749282	3802	68.571668	0.769375	230.332	0.874743	1.510000	1


In [ ]:
Data Preprocessing

In [ ]:
original_df = data_df.copy() # Creating a copy of the original Dataframe to use to normalize inference

for column in data_df.columns:
  data_df[column] = data_df[column]/data_df[column].abs().max()
data_df.head()

In [ ]:
Area	MajorAxisLength	MinorAxisLength	Eccentricity	ConvexArea	EquivDiameter	Extent	Perimeter	Roundness	AspectRation	Class
0	0.444368	0.503404	0.775435	0.744658	0.424873	0.666610	0.741661	0.537029	0.844997	0.368316	1.0
1	0.281293	0.407681	0.622653	0.750489	0.273892	0.530370	0.804230	0.409661	0.919215	0.371471	1.0
2	0.298531	0.416421	0.630442	0.756341	0.284520	0.546380	0.856278	0.412994	0.959862	0.374747	1.0
3	0.300979	0.420463	0.629049	0.764024	0.286791	0.548616	0.883772	0.414262	0.961818	0.379222	1.0
4	0.361704	0.464626	0.682901	0.775033	0.345385	0.601418	0.867808	0.452954	0.966836	0.386007	1.0


In [ ]:
Data Splitting

In [ ]:
import numpy as np# Mathematical operations
from sklearn.model_selection import train_test_split # Split the dataset (train, validation, test)

In [ ]:
X = np.array(data_df.iloc[:,:-1])# Get the inputs, all rows and all columns except last column (output)
Y = np.array(data_df.iloc[:, -1])# Get the ouputs, all rows and last column only (output column)

In [ ]:
print("Training set is: ", X_train.shape[0])
print("Training set is: ", X_test.shape[0])
print("Training set is: ", X_val.shape[0])


Training set is:  12729
Training set is:  2728
Training set is:  2728